In [1]:
import sys
sys.path.insert(0, '../../src')

import json
import random
from time import perf_counter
from collections import defaultdict

from farkle.logic import gameobjects as go

In [2]:
# for saving dicehands as json
class DiceHandEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, go.DiceHand):
            return obj.json_encode
        return super().default(obj)
    
def as_dicehand(dct):
    if '__DiceHand__' in dct:
        return go.DiceHand.json_decode(dct)
    return dct

def hash_frequencies(possible_score_frequencies):
    hash_to_dh = {}
    hash_to_freq = {}
    for d in possible_score_frequencies.keys():
        hash_to_freq[d] = {}
        for dh, freq in possible_score_frequencies[d].items():
            hash_to_dh[str(hash(dh))] = dh
            hash_to_freq[d][str(hash(dh))] = freq
    return hash_to_dh, hash_to_freq

def unhash_frequencies(hash_to_dh, hash_to_freq):
    _possible_score_freq = {}
    for d in hash_to_freq:
        d_int = int(d)
        _possible_score_freq[d_int] = defaultdict(int)
        for h, freq in hash_to_freq[d].items():
            if hash_to_dh[h] is not None:
                dh = go.DiceHand.json_decode(hash_to_dh[h])
            else:
                dh = None
            _possible_score_freq[d_int][dh] = int(freq)
    return _possible_score_freq

def json_encode(possible_score_frequencies):
    hash_to_dh, hash_to_freq = hash_frequencies(possible_score_frequencies)
    d = {'hash_to_dh': hash_to_dh, 
         'hash_to_freq': hash_to_freq}
    return json.dumps(d, cls=DiceHandEncoder)

def save_freq(path: str, possible_score_frequencies: dict):
    j = json_encode(possible_score_frequencies)
    with open(path, 'w') as f:
        f.write(j)

def load_freq(path: str):
    with open(path, 'r') as f:
        j = json.loads(f.read())

    return unhash_frequencies(j['hash_to_dh'], j['hash_to_freq'])

In [3]:
freq_path = '../../../models/possible_score_frequencies.json'
possible_score_frequencies = load_freq(freq_path)

In [4]:
possible_score_frequencies[2]

defaultdict(int,
            {DiceHand(free=[1, 1], locked=[], score=200): 1,
             DiceHand(free=[1], locked=[], score=100): 11,
             DiceHand(free=[5], locked=[], score=50): 11,
             DiceHand(free=[1, 5], locked=[], score=150): 2,
             None: 16,
             DiceHand(free=[5, 5], locked=[], score=100): 1})

In [5]:
def p_farkle(num_dice: int = None):
    if num_dice == 0: num_dice = 6
    if num_dice is not None:
        return possible_score_frequencies[num_dice][None] / 6**num_dice
    else:
        return {num_dice: possible_score_frequencies[num_dice][None] / 6**num_dice 
                for num_dice in possible_score_frequencies}

In [6]:
p_farkle(0)

0.030864197530864196

In [7]:
p_farkle()

{1: 0.6666666666666666,
 2: 0.4444444444444444,
 3: 0.2777777777777778,
 4: 0.1574074074074074,
 5: 0.07716049382716049,
 6: 0.030864197530864196}

In [24]:
dh = go.DiceHand(2,2,2,1,4, score=100)
dh.possible_scores()

[DiceHand(free=[1, 2, 2, 2], locked=[], score=300),
 DiceHand(free=[1], locked=[], score=100),
 DiceHand(free=[2, 2, 2], locked=[], score=200)]

In [25]:
for ps in dh.possible_scores():
    ps: go.DiceHand
    added_score = ps.score
    total_score = dh.score + added_score
    dice_left = len(dh.free_dice) - len(ps.free_dice)
    print(ps)
    print(f'Total score: {total_score}')
    print(f'Dice remaining: {dice_left}')
    print(f'Prob of farkle: {p_farkle(dice_left)}')
    print(f'Weighted score w/o 50: {(1-p_farkle(dice_left)) * total_score}')
    print(f'Weighted score: {(1-p_farkle(dice_left)) * (total_score+50)}')
    print('\n')

DiceHand(free=[1, 2, 2, 2], locked=[], score=300)
Total score: 400
Dice remaining: 1
Prob of farkle: 0.6666666666666666
Weighted score w/o 50: 133.33333333333334
Weighted score: 150.00000000000003


DiceHand(free=[1], locked=[], score=100)
Total score: 200
Dice remaining: 4
Prob of farkle: 0.1574074074074074
Weighted score w/o 50: 168.5185185185185
Weighted score: 210.64814814814815


DiceHand(free=[2, 2, 2], locked=[], score=200)
Total score: 300
Dice remaining: 2
Prob of farkle: 0.4444444444444444
Weighted score w/o 50: 166.66666666666669
Weighted score: 194.44444444444446




Weighted score is $(1-P[farkle]) \times (TotalScore + 50)$ since if you don't farkle on the next turn you will receive at least 50 points. This gives the minimum expected value if you take that scoring combination and choose to roll again. 

From messing around with it it seems to give higher weighted scores to choices that I personally make in game play. 